In [1]:
# !pip install tensorflow

In [2]:
import json
import os
import fnmatch
import pandas as pd
import seaborn
import numpy as np

import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt

Instructions for updating:
non-resource variables are not supported in the long term


# Unzip NASLib/run/results/bananas_400.zip

In [3]:
BENCHMARKS = {
#     'nasbench101': ['cifar10'],
    'nasbench201': ['cifar10', 'cifar100', 'ImageNet16-120'],
    'nasbench301': ['cifar10'],
    'transbench101_micro': ['jigsaw', 'class_scene', 'class_object', 'autoencoder', 'normal', 'room_layout', 'segmentsemantic'],
    'transbench101_macro': ['jigsaw', 'class_scene', 'class_object', 'autoencoder', 'normal', 'room_layout', 'segmentsemantic']
}

LABELS = {
    'nasbench101': 'NB101',
    'nasbench201': 'NB201',
    'nasbench301': 'NB301',
    'transbench101_micro': 'TNB101_MICRO',
    'transbench101_macro': 'TNB101_MACRO',
    'cifar10': 'CF10',
    'cifar100': 'CF100',
    'ImageNet16-120': 'IMGNT',
    'jigsaw': 'JIGSAW',
    'class_scene': 'SCENE',
    'class_object': 'OBJECT',
    'autoencoder': 'AUTOENC',
    'normal': 'NORMAL',
    'room_layout': 'ROOM',
    'segmentsemantic': 'SEGMENT',
}

START_SEED = 9000
N_SEEDS = 32
N_EPOCHS = 500

def find_files(src='../run/results/only_zc', fname='*events.out.tfevents.*'):
    matches = []
    for root, dirnames, filenames in os.walk(src):
        for filename in fnmatch.filter(filenames, fname):
            matches.append(os.path.join(root, filename))

    return matches

def read_file(event_file, tag):
    vals = []
    for e in tf.train.summary_iterator(event_file):
        for v in e.summary.value:
            if v.tag == tag:
                vals.append(v.simple_value)
    return vals

In [4]:
all_files = find_files()

In [5]:
all_files[0], len(all_files)

('../run/results/only_zc/transbench101_micro/jigsaw/zc/9009/fisher-flops-grad_norm-grasp-jacov-l2_norm-nwot-params-plain-snip-zen/events.out.tfevents.1654634872.kisexe12.917366.0',
 392)

In [6]:
def get_scores_as_list_of_dict(files):
    data = []

    for file in files:
        components = file.split('/')
        zc_names, seed, dataset, search_space = components[-2], components[-3], components[-5], components[-6]

        val_accuracies = read_file(file, "Validation accuracy (top 1)")
        train_accuracies = read_file(file, "Train accuracy (top 1)")
        
        if len(val_accuracies) == 0 or len(train_accuracies) == 0:
            continue
        
        record = {
            'search_space': search_space,
            'dataset': dataset,
            'seed': seed,
            'zc_names': zc_names,
            'val_accs': val_accuracies,
            'train_accs': train_accuracies,
        }

        data.append(record)

    return data

def make_df(files):
    data = get_scores_as_list_of_dict(files)
    df = pd.DataFrame(data)
    return df

def clean_df(df):
    bad_indices = []

    for idx, val_acc in enumerate(df['val_accs']):
        if len(val_acc) != N_EPOCHS:
            bad_indices.append(idx)

    df = df.drop(index=bad_indices)
    return df

def get_val_train_accs(df, search_space, dataset):
    df_ = df[(df['search_space'] == search_space) & (df['dataset'] == dataset)]
    
    seed_val_accs = []
    seed_train_accs = []

    for seed in range(START_SEED, START_SEED + N_SEEDS):
        try:
            vals = df_[df_['seed'] == str(seed)]['val_accs'].item()
            train_accs = df_[df_['seed'] == str(seed)]['train_accs'].item()
            seed_val_accs.append(vals)
            seed_train_accs.append(train_accs)
        except:
            print(df_[df_['seed'] == str(seed)]['val_accs'])
    
    return np.array(seed_val_accs), np.array(seed_train_accs)

def plot(x, title, xlabel, ylabel, figsize=(18, 12)):
    
    x_mean = np.mean(x, axis=0)
    x_std = np.std(x, axis=0)
    
    plt.figure(figsize=figsize, dpi=200)
    plt.title(title, fontsize=20)
    plt.plot(x_mean)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)

    plt.fill_between(range(len(x_mean)), x_mean+x_std, x_mean-x_std, alpha=0.6)


def plot_multiple(x_only_zc, x_only_adj, x_zc, title, xlabel, ylabel, figsize=(18, 12)):
    plt.figure(figsize=figsize, dpi=200)
    plt.title(title, fontsize=20)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)

    titles = ('ONLY_ZC', 'ONLY_ADJ', 'BOTH')
    
    for idx, x in enumerate((x_only_zc, x_only_adj, x_zc)):
        x_mean = np.mean(x, axis=0)
        x_std = np.std(x, axis=0)
        plt.plot(x_mean, label=titles[idx])
        plt.fill_between(range(len(x_mean)), x_mean+x_std, x_mean-x_std, alpha=0.4)

    plt.legend()

def plot_for_search_space(df, search_space):
    for dataset in BENCHMARKS[search_space]:
        val_accs, train_accs = get_val_train_accs(df, search_space, dataset)
        plot(val_accs, f'{LABELS[search_space]} {LABELS[dataset]}', 'Epochs', 'Validation Accuracy (Top 1)')

def plot_search_space(search_space, only_zc_df, only_adj_df, zc_df):    
    for dataset in BENCHMARKS[search_space]:
        only_zc_vals, _ = get_val_train_accs(only_zc_df, search_space, dataset)
        only_adj_vals, _ = get_val_train_accs(only_adj_df, search_space, dataset)
        zc_vals, _ = get_val_train_accs(zc_df, search_space, dataset)

        plot_multiple(only_zc_vals, only_adj_vals, zc_vals, f'{LABELS[search_space]} {LABELS[dataset]}', 'Epochs', 'Validation Accuracy (Top 1)' )

def save_df(df, name):
    df.to_pickle(name)

def load_df(name):
    return pd.read_pickle(name)

In [7]:
# df = make_df(all_files)
# plot_for_search_space(df, 'transbench101_macro')
# plot_for_search_space(df, 'nasbench301')
# plot_for_search_space(df, 'nasbench201')

In [8]:
only_zc_files = find_files(src='../run/results/only_zc')
only_adj_files = find_files(src='../run/results/only_adjacency')
zc_files = find_files(src='../run/results/zc_and_adjacency')

len(only_zc_files), len(only_adj_files), len(zc_files)

(392, 392, 392)

In [ ]:
only_zc_df = make_df(only_zc_files)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [ ]:
only_adj_df = make_df(only_adj_files)

In [ ]:
zc_df = make_df(zc_files)

In [ ]:
# save_df(only_zc_df, 'only_zc.pkl')
# save_df(only_adj_df, 'only_adj.pkl')
# save_df(zc_df, 'zc_and_adj.pkl')

In [ ]:
len(only_adj_df)

In [ ]:
only_zc_df = clean_df(only_zc_df)
only_adj_df = clean_df(only_adj_df)
zc_df = clean_df(zc_df)

len(only_zc_df), len(only_adj_df), len(zc_df)

In [ ]:
def plot_multiple(x_only_zc, x_only_adj, x_zc, title, xlabel, ylabel, figsize=(6, 4)):
    plt.figure(figsize=figsize, dpi=200)
    plt.title(title, fontsize=20)
    plt.xlabel(xlabel)
    if 'AUTOENC' in title or 'NORMAL' in title:
        ylabel = 'SSIM'
    elif 'SEGMENT' in title:
        ylabel = 'mIoU'
    elif 'ROOM' in title:
        ylabel = 'L2 Loss'
    else:
        ylabel = 'Accuracy (%)'
    plt.ylabel(ylabel)
    plt.xscale('log')
    #plt.yscale('log')
    plt.grid(True)

    titles = ('Encoding', 'ZCPs', 'Encoding + ZCPs')
    
    os.makedirs('bananas', exist_ok=True)
    
    title_to_lim = {
        'NB201 CF10': [90, 92],
        'NB201 CF100': [71, 73],
        'NB201 IMGNT': [46, 46.6],
        'NB301 CF10': [94, 94.6],
        'TNB101_MICRO JIGSAW': [94.5, 95],
        'TNB101_MICRO SCENE': [54.25, 54.9],
        'TNB101_MICRO OBJECT': [44.75, 45.75],
        'TNB101_MICRO AUTOENC': [0.54, 0.58],
        'TNB101_MICRO NORMAL': [0.56, 0.58],
        'TNB101_MICRO ROOM': [-0.625, -0.6],
        'TNB101_MICRO SEGMENT': [94.6, 94.8],
        'TNB101_MACRO JIGSAW': [96.5, 97],
        'TNB101_MACRO SCENE': [56.5, 57],
        'TNB101_MACRO OBJECT': [47, 47.6],
        'TNB101_MACRO AUTOENC': [0.7, 0.78],
        'TNB101_MACRO NORMAL': [0.62, 0.65],
        'TNB101_MACRO ROOM': [-0.6, -0.56],
        'TNB101_MACRO SEGMENT': [94.95, 95.1],   
    }
    plt.ylim(title_to_lim[title])
    plt.xlim([10, 200])
    
    for idx, x in enumerate((x_only_adj, x_only_zc, x_zc)):
#         print(idx, x)
        x_mean = np.mean(x, axis=0)
        print(idx, x_mean.shape)
        x_std = np.std(x, axis=0)/np.sqrt(10)
        plt.plot(x_mean, label=titles[idx])
        plt.fill_between(range(len(x_mean)), x_mean+x_std, x_mean-x_std, alpha=0.2)

    plt.legend()
    plt.savefig('bananas/{}.pdf'.format(title.replace(' ', '_')), bbox_inches='tight')

plot_search_space('nasbench201', only_zc_df, only_adj_df, zc_df)
plot_search_space('nasbench301', only_zc_df, only_adj_df, zc_df)
plot_search_space('transbench101_micro', only_zc_df, only_adj_df, zc_df)
# plot_search_space('transbench101_macro', only_zc_df, only_adj_df, zc_df)

In [ ]:
only_zc_df[:100]

In [ ]:
only_adj_df[:100]

In [ ]:
result = np.mean(only_adj_df, axis=0)
result[:10]

In [ ]:
only_zc_df['val_accs']